# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
df = pd.read_csv(r'C:\Users\zr_1\python-api-challenge\output_data\output_data_file.csv', encoding="utf-8")
df.head()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

locations = df[['lat', 'lng']]
humidity = df['humidity'].astype(float)
maxhumidity = humidity.max()

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False, max_intensity=maxhumidity,point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_df = df.loc[(df["temperature"] > 70) & (df["temperature"] < 80) & (df["clouds"] == 0) & (df["wind speed"] < 10), :]
narrowed_df = narrowed_df.dropna(how='any')
narrowed_df.reset_index(inplace=True)
narrowed_df.head()

,index,Unnamed: 0,city id,city,country,lat,lng,temperature,humidity,clouds,wind speed
0,23,23,2455290,Kidal,ML,18.4411,1.4078,70.48,18,0,8.23
1,141,141,2447513,Arlit,NE,18.7369,7.3853,74.84,16,0,8.66
2,190,190,4021858,Guerrero Negro,MX,27.9769,-114.0611,71.38,22,0,2.44
3,303,303,2381972,Adrar,MR,20.5022,-10.0711,70.27,21,0,9.04
4,309,309,1260771,Palāsa,IN,18.7667,84.4167,72.45,69,0,6.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = []

for x in range(len(narrowed_df)):
    lat = narrowed_df.loc[x]['lat']
    lng = narrowed_df.loc[x]['lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    request = requests.get(base_url, params=params)
    json = request.json()
    try:
        hotel_df.append(jsn['results'][0]['name'])
    except:
        hotel_df.append("")
narrowed_df["Hotel Name"] = hotel_df
narrowed_df = narrowed_df.dropna(how='any')
narrowed_df.head()

{'error_message': 'You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started',
 'html_attributions': [],
 'results': [],
 'status': 'REQUEST_DENIED'}

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
